In [1]:
from requests import get, post
import pandas as pd
import numpy as np
import sympy as sp
from matplotlib import pyplot as plt

# Carga de Dados
[Origem dos Dados](https://github.com/binance/binance-spot-api-docs/blob/master/rest-api.md#klinecandlestick-data)

In [2]:
# Endpoints
url = 'https://api3.binance.com'
klines = url + '/api/v3/klines'
info = url + '/api/v3/exchangeInfo'

In [3]:
# Colunas
col = [
    'Open time','Open','High','Low','Close','Volume','Close time',
    'Quote asset volume','Number of trades','Taker buy base asset volume',
    'Taker buy quote asset volume','Ignore'
]

# Parametros
params = {
    "btc": {
    "symbol": "BTCUSDT",
    "interval": "1d",
    "limit": 1000
    },
    "eth": { 
    "symbol": "ETHUSDT",
    "interval": "1d",
    "limit": 1000
    },"ltc": { 
    "symbol": "LTCUSDT",
    "interval": "1d",
    "limit": 1000
    }
}

In [57]:
def clean(x):    
    # Obtendo dados
    asset = get(klines, params=params[x]).json()

    # Limando - > deixando apenas as colunas: (close, time_close)
    asset = np.array(asset, dtype=float)[..., 4:8:2]
    
    return asset

arr = np.concatenate([*map(clean, params)], axis=1)

df = pd.DataFrame(arr[..., 0::2], index=arr[..., 1], columns=params.keys())

# Calculando Taxa de Retorno Simples
$$
\frac{P_i - P_f}{P_i} = \frac{P_f}{P_i} - 1
$$

In [59]:
df['btc_return'] = (df.btc / df.btc.shift(1)) - 1
df['eth_return'] = (df.eth / df.eth.shift(1)) - 1
df['ltc_return'] = (df.ltc / df.ltc.shift(1)) - 1

In [71]:
f = lambda x: 'color: red' if x < 0 else 'color: green'
df.head(10).style.applymap(f, subset=['btc_return', 'eth_return', 'ltc_return'])

,btc,eth,ltc,btc_return,eth_return,ltc_return
1528847999999.0,6530.000000,493.000000,99.680000,nan,nan,nan
1528934399999.0,6292.780000,475.470000,93.370000,-0.037004,-0.035558,-0.063303
1529020799999.0,6635.980000,519.290000,100.880000,0.053103,0.092161,0.080433
1529107199999.0,6388.900000,487.480000,95.500000,-0.037944,-0.061257,-0.053331
1529193599999.0,6483.980000,496.740000,96.660000,0.014772,0.018996,0.012147
1529279999999.0,6449.610000,497.140000,94.950000,-0.005315,0.000805,-0.017691
1529366399999.0,6712.460000,517.950000,98.700000,0.039946,0.041859,0.039494
1529452799999.0,6741.210000,537.970000,98.670000,0.004274,0.038652,-0.000304
1529539199999.0,6761.510000,536.490000,97.720000,0.003007,-0.002751,-0.009628
1529625599999.0,6718.840000,526.180000,96.720000,-0.006331,-0.019218,-0.010233


# Volatilidade usando Retornos Logaritmos

$$
log(\frac{P_t}{P_{t-1}})
$$

In [68]:
# Calculo de Retorno
df['btc_return'] = np.log(df.btc / df.btc.shift(1))
df['btc_return'] = np.log(df.btc / df.btc.shift(1))
df['btc_return'] = np.log(df.btc / df.btc.shift(1))

In [72]:
f = lambda x: 'color: red' if x < 0 else 'color: green'
df.head(10).style.applymap(f, subset=['btc_return', 'eth_return', 'ltc_return'])

,btc,eth,ltc,btc_return,eth_return,ltc_return
1528847999999.0,6530.000000,493.000000,99.680000,nan,nan,nan
1528934399999.0,6292.780000,475.470000,93.370000,-0.037004,-0.035558,-0.063303
1529020799999.0,6635.980000,519.290000,100.880000,0.053103,0.092161,0.080433
1529107199999.0,6388.900000,487.480000,95.500000,-0.037944,-0.061257,-0.053331
1529193599999.0,6483.980000,496.740000,96.660000,0.014772,0.018996,0.012147
1529279999999.0,6449.610000,497.140000,94.950000,-0.005315,0.000805,-0.017691
1529366399999.0,6712.460000,517.950000,98.700000,0.039946,0.041859,0.039494
1529452799999.0,6741.210000,537.970000,98.670000,0.004274,0.038652,-0.000304
1529539199999.0,6761.510000,536.490000,97.720000,0.003007,-0.002751,-0.009628
1529625599999.0,6718.840000,526.180000,96.720000,-0.006331,-0.019218,-0.010233


### Análise do BTC, ETH e LTC

In [115]:
for i in df.iloc[:, 3:]:
    print(f'\033[31mRetorno Médio: {i}\033[m')
    print(df[i].mean())
    
    print(f'\033[31mRetorno Anual 360d: {i}\033[m')
    print(df[i].mean() * 365)

    print(f'\033[31mVolatilidade Média: {i}\033[m')
    print(df[i].std())

    print(f'\033[31mVolatilidade Anual 365: {i}\033[m')
    print(df[i].std() * 365 ** 0.5, end='\n\n')

Retorno Médio: btc_return
0.002029874618132326
Retorno Anual 360d: btc_return
0.7409042356182989
Volatilidade Média: btc_return
0.03934809795800291
Volatilidade Anual 365: btc_return
0.7517443559569279

Retorno Médio: eth_return
0.002518692027207086
Retorno Anual 360d: eth_return
0.9193225899305864
Volatilidade Média: eth_return
0.05032609203249631
Volatilidade Anual 365: eth_return
0.961478638260414

Retorno Médio: ltc_return
0.0019844977514809595
Retorno Anual 360d: ltc_return
0.7243416792905503
Volatilidade Média: ltc_return
0.05207671598829915
Volatilidade Anual 365: ltc_return
0.9949242619747394



In [116]:
def f(x):
    x = df[x].mean()
    
    y = df[x].mean() * 365

    z = df[x].std()

    w = df[x].std() * 365 ** 0.5
    return [x,y,z,w]